In [85]:
import pandas as pd
import configparser
from sqlalchemy import create_engine, text

In [87]:
from mysql.connector import MySQLConnection, Error

In [89]:
#Connecting to the shared SQL Server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [91]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [93]:
# Load CSV file
csv_file = "/Users/kaushikhariharan/Desktop/Data201/Assignments/TheQueryCrew/Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(csv_file)

# Extracting 50 rows of "Make" and "Model Year" from CSV (using .loc for label indexing the CSV file)
#We only want the UNIQUE values of Make to present the One-to-Many Relationship with the second table.
make_and_model_year_data = df.loc[:49, ["Make", "Model Year"]].groupby("Make").first().reset_index()

# Extracting 50 rows of "Make", "Model", and "Vehicle Type" from CSV file.
make_model_type_data = df.loc[:49, ["Make", "Model", "Electric Vehicle Type"]]

print("Make and Model Year Data")
print(make_and_model_year_data.head(25))
 
print("\n Make Model Type Data")
print(make_model_type_data.head(25))

Make and Model Year Data
         Make  Model Year
0         BMW        2015
1   CHEVROLET        2019
2        FIAT        2015
3     HYUNDAI        2022
4        JEEP        2023
5         KIA        2024
6       LEXUS        2023
7       MAZDA        2024
8      NISSAN        2017
9    POLESTAR        2021
10     RIVIAN        2022
11      TESLA        2019
12     TOYOTA        2018
13      VOLVO        2022

 Make Model Type Data
         Make     Model                   Electric Vehicle Type
0        JEEP  WRANGLER  Plug-in Hybrid Electric Vehicle (PHEV)
1   CHEVROLET   BOLT EV          Battery Electric Vehicle (BEV)
2         BMW        I3          Battery Electric Vehicle (BEV)
3       TESLA   MODEL 3          Battery Electric Vehicle (BEV)
4       TESLA   MODEL S          Battery Electric Vehicle (BEV)
5        JEEP  WRANGLER  Plug-in Hybrid Electric Vehicle (PHEV)
6         BMW      530E  Plug-in Hybrid Electric Vehicle (PHEV)
7       TESLA   MODEL X          Battery Electric 

In [95]:
#Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS make_model_year')
cursor.execute('DROP TABLE IF EXISTS make_model_type')

#Creating the tables to hold make, year, model, and type of vehicles
make_model_year = ( """
        CREATE TABLE make_model_year
        (
            make       varchar(32),
            year      int,
            PRIMARY KEY (make)
        )
        """
      )
make_model_type = ( """
        CREATE TABLE make_model_type
        (
            make       varchar(32),
            model   varchar(32),
            type        varchar(64)
        )
        """
      )

cursor.execute(make_model_year);
cursor.execute(make_model_type);

print("Tables created successfully!")

Tables created successfully!


In [97]:
#Inserting values into the respective tables
sql_insert_make_model_year = (  
    """
    INSERT INTO make_model_year (make, year)
    VALUES (%s, %s)
    """
)

sql_insert_make_model_type = (  
    """
    INSERT INTO make_model_type (make, model, type)
    VALUES (%s, %s, %s)
    """
)

#Looping and iterating through the respective DataFrames to insert values into created Databases.
for row in make_and_model_year_data.itertuples(index=False, name=None):
    cursor.execute(sql_insert_make_model_year, row)
print("Inserted make and model_year data successfully!")
for row in make_model_type_data.itertuples(index=False, name=None):
    cursor.execute(sql_insert_make_model_type, row)
print("Inserted make, model, and type successfully!")


Inserted make and model_year data successfully!
Inserted make, model, and type successfully!


In [99]:
#Displaying the one-to-many relationship
#where one MAKE can have MULTIPLE MODELS

print("Make-Model-Year Table")
sql_make_model_year = (
    """
    SELECT * FROM make_model_year LIMIT 25;
    """
)
cursor.execute(sql_make_model_year)
result_data = cursor.fetchall()
for r in result_data:
    print(r)

print("\nMake-Model-Type Table")
sql_make_model_type = (
    """
    SELECT * FROM make_model_type LIMIT 25;
    """
)
cursor.execute(sql_make_model_type)
result_data_2 = cursor.fetchall()
for r2 in result_data_2:
    print(r2)


Make-Model-Year Table
('BMW', 2015)
('CHEVROLET', 2019)
('FIAT', 2015)
('HYUNDAI', 2022)
('JEEP', 2023)
('KIA', 2024)
('LEXUS', 2023)
('MAZDA', 2024)
('NISSAN', 2017)
('POLESTAR', 2021)
('RIVIAN', 2022)
('TESLA', 2019)
('TOYOTA', 2018)
('VOLVO', 2022)

Make-Model-Type Table
('JEEP', 'WRANGLER', 'Plug-in Hybrid Electric Vehicle (PHEV)')
('CHEVROLET', 'BOLT EV', 'Battery Electric Vehicle (BEV)')
('BMW', 'I3', 'Battery Electric Vehicle (BEV)')
('TESLA', 'MODEL 3', 'Battery Electric Vehicle (BEV)')
('TESLA', 'MODEL S', 'Battery Electric Vehicle (BEV)')
('JEEP', 'WRANGLER', 'Plug-in Hybrid Electric Vehicle (PHEV)')
('BMW', '530E', 'Plug-in Hybrid Electric Vehicle (PHEV)')
('TESLA', 'MODEL X', 'Battery Electric Vehicle (BEV)')
('CHEVROLET', 'BOLT EV', 'Battery Electric Vehicle (BEV)')
('TESLA', 'MODEL 3', 'Battery Electric Vehicle (BEV)')
('NISSAN', 'LEAF', 'Battery Electric Vehicle (BEV)')
('LEXUS', 'RZ', 'Battery Electric Vehicle (BEV)')
('KIA', 'EV6', 'Battery Electric Vehicle (BEV)')
('N